In [1]:
!pip install tiktoken
!pip install plotly
!pip install scipy
!pip install scikit-learn

  Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl (798 kB)


In [2]:
!pip install openai

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/311.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/311.2 kB ? eta -:--:--
   --- ----------------------------------- 30.7/311.2 kB 330.3 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/311.2 kB 655.4 kB/s eta 0:00:01
   ---------------------------------------- 311.2/311.2 kB 1.7 MB/s eta 0:00:00
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [3]:
import pandas as pd
import re

In [4]:
def remove_newlines(text):
    #正規表現で文字列を置換
    #改行と連続する空白を半角スペースに変換
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text

In [5]:
def text_to_df(data_file):
    #
    texts =[]
    #utf-8でテキストファイルを開く
    with open(data_file, 'r', encoding='utf-8') as file:
        #すべて読み込む
        text = file.read()
        #改行２つを区切りに分割
        sections = text.split('\n\n')
        #項目ごと
        for section in sections:
            #改行で分割
            lines = section.split('\n')
            #配列最初の要素に項目名（例：1. ゲストの歓迎）
            fname = lines[0]
            #配列１番目以降の要素すべてをつなげる
            content = ' '.join(lines[1:])
            #配列を追加
            texts.append([fname, content])
    #配列を元にデータフレーム作成
    df = pd.DataFrame(texts, columns=['fname', 'text'])
    #列内の改行を削除
    df['text'] = df['text'].apply(remove_newlines)
    #データフレームを返す
    return df

In [6]:
#データ読み込み
df = text_to_df('data.txt')
#表をCSVに保存
df.to_csv('scraped.csv', index=False, encoding='utf-8')

In [7]:
df

,fname,text
0,1. ゲストの歓迎,お客様がホテルに到着した際、フレンドリーな笑顔と共に、礼儀正しく、エネルギッシュな挨拶を心掛...
1,2. チェックインとチェックアウト,チェックイン時間は午後3時、チェックアウト時間は午前11時です。早めのチェックインや遅めのチ...
2,3. Wi-Fiと駐車場の案内,全室に無料Wi-Fiが提供されています。接続方法やパスワードについて、確実に説明できるように...
3,4. バリアフリー対応,ユニバーサルルームの配置や設備、特長を理解し、必要に応じてお客様に説明できるようにしましょう...
4,5. ペットの対応,ペット同伴のお客様に対しては、礼儀正しく、しかし明確にペットの同伴はできない旨を伝えましょう...
5,6. ルームサービス,午後11時までのルームサービスを提供しています。ルームサービスメニューの内容を熟知し、お客様...
6,7. 禁煙ポリシーと喫煙室の案内,全客室は禁煙です。しかし、喫煙者のお客様のニーズにも応えるため、喫煙室を1階に設けています。...
7,8. キャンセルポリシー,キャンセル料は、前日までの連絡で宿泊料金の30%、当日のキャンセルで50%、連絡なしの場合は...
8,9. お支払いについて,チェックアウト時にはフロントで現金、クレジットカード、デビットカードによるお支払いをお願いし...
9,10. 常に敬意を持つ,お客様一人一人に敬意を持って接しましょう。お客様に対する礼儀正しさ、思いやり、プロ意識は、ホ...


### 学習データをエンベディング

In [14]:
#!pip uninstall openai
!pip install openai==0.28.1

In [21]:
!pip install matplotlib

In [8]:
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
import tiktoken
#from openai.embeddings_utils import get_embedding

In [7]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
max_tokens = 1500

df = pd.read_csv('scraped.csv')
df.columns = ['title', 'text']

tokenizer = tiktoken.get_encoding(embedding_encoding)
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))



In [8]:
def split_into_many(text, max_tokens = 500):
    #
    sentences = text.split('。')
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    #
    chunks = []
    tokens_so_far = 0
    chunk = []
    #
    for sentence, token in zip(sentences, n_tokens):
        #
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0
        #
        if token > max_tokens:
            continue

        chunk.append(sentence)
        tokens_so_far += token + 1

    if chunk:
        chunks.append(". ".join(chunk) + ".")
    #    
    return chunks

In [ ]:
#
shortened = []

#
for row in df.iterrows():
    #
    if row[1]['text'] is None:
        continue

    #
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])

    #
    else:
        #
        shortened.append(row[1]['text'])

#
df['n_tokens'] = df.text.apply(lambda x: get_embedding(x, engine=embedding_model))
#
df.to_csv('embedding.csv')